In [2]:
from train import create_df, build_parser, ARDSDetectionModel

args = build_parser().parse_args([])
args.frame_size = 100
args.n_new_features = 8
args.feature_selection_method = 'chi2'
args.from_pickle = 'cohort-100220.pkl'
df = create_df(args)
model = ARDSDetectionModel(args, df)

In [3]:
model.train_and_test()

----Run fold 1----
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']


/home/greg/anaconda3/envs/ardsaim1/lib/python2.7/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/greg/anaconda3/envs/ardsaim1/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: [

Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']
+-------+-----------

In [4]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(model.models[0])

In [5]:
splits = model.perform_data_splits()
splits = list(splits)

In [13]:
x_train, x_test, y_train, y_test = splits[0]
# 0 is for fold index 0. Or fold #1 if you think arrays start at 1 :P
x_test = model.perform_feature_selection(x_train, y_train, x_test, 0)
selected_features = x_test.columns
x_train = x_train[selected_features]

Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance', 'median_tve:tvi ratio']


In [14]:
explainer = shap.TreeExplainer(model.models[0])
vals = explainer.shap_values(x_train)
shap.force_plot(explainer.expected_value[0], vals[0][0,:], x_train.iloc[0,:])

In [15]:
cell = x_train.iloc[0,:].tolist()
model.models[0].predict([cell])

array([0])

In [16]:
shap.force_plot(explainer.expected_value[0], vals[0][0:100,:], x_train.iloc[0:100,:])

In [17]:
pt_112 = model.data[model.data.patient == '0112RPI1620160105']

In [18]:
explainer = shap.TreeExplainer(model.models[0])
pt_112_x = x_test.loc[pt_112.index]
pt_112_vals = explainer.shap_values(pt_112_x)

In [19]:
shap.force_plot(explainer.expected_value[1], pt_112_vals[1], pt_112_x)

In [20]:
3/308.0

0.00974025974025974

In [76]:
all_results = model.results.get_all_patient_results_dataframe()
# Obtain results from 3rd fold as an example
fold_idx = 2
all_results = all_results[all_results.fold_idx==fold_idx]
fold_3 = all_results.groupby('patient_id').mean()
fold_3.loc[fold_3.majority_prediction < 0.5, 'majority_prediction'] = 0
fold_3.loc[fold_3.majority_prediction > 0.5, 'majority_prediction'] = 1
fold_3['patient_id'] = fold_3.index
fold_3.index = range(len(fold_3))

In [108]:
# near total confidence is described as >.99 majority_prediction threshold
patients_with_near_total_confidence = fold_3[(fold_3.ground_truth != fold_3.majority_prediction) & (fold_3.frac_votes >= .99)]
patients_with_very_high_confidence = fold_3[(fold_3.ground_truth != fold_3.majority_prediction) & (fold_3.frac_votes >= .8) & (fold_3.frac_votes < .99)][['patient_id', 'model_idx']]
patients_with_high_confidence = fold_3[(fold_3.ground_truth != fold_3.majority_prediction) & (fold_3.frac_votes >= .65) & (fold_3.frac_votes < .8)][['patient_id', 'model_idx']]
incorrect_classifications = fold_3[(fold_3.ground_truth != fold_3.majority_prediction) & (fold_3.frac_votes > .5) & (fold_3.frac_votes < .65)][['patient_id', 'model_idx']]

In [109]:
patients_with_near_total_confidence.patient_id

15    0343RPI3920161016
17    0354RPI5820161029
Name: patient_id, dtype: object

In [110]:
patients_with_very_high_confidence

,patient_id,model_idx
10,0304RPI1620160829,24.5


In [111]:
patients_with_high_confidence

,patient_id,model_idx


In [112]:
incorrect_classifications

,patient_id,model_idx
0,0224RPI3020160414,24.5


In [113]:
pt = model.data[model.data.patient == patients_with_near_total_confidence.patient_id.iloc[0]]
print(pt.patient.unique()[0])
# just use the model index that can be closest in rounding. Its not perfect but good enuf.
model_idx = int(patients_with_near_total_confidence.model_idx.iloc[0])
plot_patient_sequence(model, pt, model_idx, splits, fold_idx)
explainer = shap.TreeExplainer(model.models[model_idx])
x_train, x_test, y_train, y_test = splits[fold_idx]
x_test = model.perform_feature_selection(x_train, y_train, x_test, fold_idx)

pt_x = x_test.loc[pt.index]
pt_vals = explainer.shap_values(pt_x)
features = model.data.loc[pt_x.index][x_test.columns]
shap.force_plot(explainer.expected_value[1], pt_vals[1], features)

0343RPI3920161016
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_slope_minF_to_zero', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance']
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_slope_minF_to_zero', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance']


In [114]:
idx = 1
pt = model.data[model.data.patient == patients_with_near_total_confidence.patient_id.iloc[idx]]
print(pt.patient.unique()[0])
model_idx = int(patients_with_near_total_confidence.model_idx.iloc[idx])
explainer = shap.TreeExplainer(model.models[model_idx])
x_train, x_test, y_train, y_test = splits[fold_idx]
x_test = model.perform_feature_selection(x_train, y_train, x_test, fold_idx)

pt_x = x_test.loc[pt.index]
pt_vals = explainer.shap_values(pt_x)
features = model.data.loc[pt_x.index][x_test.columns]
shap.force_plot(explainer.expected_value[1], pt_vals[1], features)

0354RPI5820161029
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_slope_minF_to_zero', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance']


In [115]:
idx = 0
pt = model.data[model.data.patient == patients_with_very_high_confidence.patient_id.iloc[idx]]
print(pt.patient.unique()[0])
model_idx = int(patients_with_near_total_confidence.model_idx.iloc[idx])
explainer = shap.TreeExplainer(model.models[model_idx])
x_train, x_test, y_train, y_test = splits[fold_idx]
x_test = model.perform_feature_selection(x_train, y_train, x_test, fold_idx)

pt_x = x_test.loc[pt.index]
pt_vals = explainer.shap_values(pt_x)
features = model.data.loc[pt_x.index][x_test.columns]
shap.force_plot(explainer.expected_value[1], pt_vals[1], features)

0304RPI1620160829
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_slope_minF_to_zero', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance']


In [117]:
idx = 0
pt = model.data[model.data.patient == incorrect_classifications.patient_id.iloc[idx]]
print(pt.patient.unique()[0])
model_idx = int(patients_with_near_total_confidence.model_idx.iloc[idx])
explainer = shap.TreeExplainer(model.models[model_idx])
x_train, x_test, y_train, y_test = splits[fold_idx]
x_test = model.perform_feature_selection(x_train, y_train, x_test, fold_idx)

pt_x = x_test.loc[pt.index]
pt_vals = explainer.shap_values(pt_x)
features = model.data.loc[pt_x.index][x_test.columns]
shap.force_plot(explainer.expected_value[1], pt_vals[1], features)

0224RPI3020160414
Selected features: ['median_mean_flow_from_pef', 'median_inst_RR', 'median_slope_minF_to_zero', 'median_pef_+0.16_to_zero', 'median_iTime', 'median_eTime', 'median_I:E ratio', 'median_dyn_compliance']
